In [1]:
# Importing the libraries 

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

# Ignore harmless warnings 

import warnings 
warnings.filterwarnings("ignore")

# Set to display all the columns in dataset

pd.set_option("display.max_columns", None)

# Import psql to run queries 

import pandasql as psql

In [2]:
concrete = pd.read_csv(r"C:\Users\badda\Downloads\Concrete_Data_V2.0.csv",header = 0)

concrete.head()

,Test_ID,Cement,Slag,Flyash,Water,SuperPlasticizer,CoarseAggregate,FineAggregate,Age,CSinMPa
0,C3417,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,P7675,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,Y5697,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,R3821,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,P3773,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
concrete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Test_ID           1030 non-null   object 
 1   Cement            1030 non-null   float64
 2   Slag              1030 non-null   float64
 3   Flyash            1030 non-null   float64
 4   Water             1030 non-null   float64
 5   SuperPlasticizer  1030 non-null   float64
 6   CoarseAggregate   1030 non-null   float64
 7   FineAggregate     1030 non-null   float64
 8   Age               1030 non-null   int64  
 9   CSinMPa           1030 non-null   float64
dtypes: float64(8), int64(1), object(1)
memory usage: 80.6+ KB


In [4]:
concrete_dup =concrete[concrete.duplicated(keep = 'last')]

concrete_dup

,Test_ID,Cement,Slag,Flyash,Water,SuperPlasticizer,CoarseAggregate,FineAggregate,Age,CSinMPa


In [5]:
concrete_BK1 = concrete.copy()

In [6]:
concrete.nunique()

Test_ID             1028
Cement               278
Slag                 185
Flyash               156
Water                195
SuperPlasticizer     111
CoarseAggregate      284
FineAggregate        302
Age                   14
CSinMPa              845
dtype: int64

In [7]:
concrete.isnull().sum()

Test_ID             0
Cement              0
Slag                0
Flyash              0
Water               0
SuperPlasticizer    0
CoarseAggregate     0
FineAggregate       0
Age                 0
CSinMPa             0
dtype: int64

In [8]:
concrete.shape

(1030, 10)

In [9]:
#eliminate the outliers in 'CsinMPa' and write data to new file

CSinMPa_UL = round(concrete.CSinMPa.mean()+3 * concrete.CSinMPa.std(),3)
CSinMPa_LL = round(concrete.CSinMPa.mean()-3 * concrete.CSinMPa.std(),3)
concrete_new = concrete[(concrete.CSinMPa > CSinMPa_LL)&(concrete.CSinMPa  < CSinMPa_UL)]

concrete_bk = concrete_new

concrete_new.shape

(1030, 10)

In [10]:
del concrete_new['Test_ID']


concrete_new.head()

,Cement,Slag,Flyash,Water,SuperPlasticizer,CoarseAggregate,FineAggregate,Age,CSinMPa
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [11]:
concrete_new.describe().T   #Statistical information of our dataframe

,count,mean,std,min,25%,50%,75%,max
Cement,1030.0,281.167864,104.506364,102.00,192.375,272.900,350.000,540.0
Slag,1030.0,73.895825,86.279342,0.00,0.000,22.000,142.950,359.4
Flyash,1030.0,54.188350,63.997004,0.00,0.000,0.000,118.300,200.1
Water,1030.0,181.567282,21.354219,121.80,164.900,185.000,192.000,247.0
SuperPlasticizer,1030.0,6.204660,5.973841,0.00,0.000,6.400,10.200,32.2
CoarseAggregate,1030.0,972.918932,77.753954,801.00,932.000,968.000,1029.400,1145.0
FineAggregate,1030.0,773.580485,80.175980,594.00,730.950,779.500,824.000,992.6
Age,1030.0,45.662136,63.169912,1.00,7.000,28.000,56.000,365.0
CSinMPa,1030.0,35.817961,16.705742,2.33,23.710,34.445,46.135,82.6


In [12]:
#identifying the independent and dependent variable

IndepVar = []

for col in concrete_new.columns:
    if col !='CSinMPa':
        IndepVar.append(col)
        
TargetVar = 'CSinMPa'

x = concrete_new[IndepVar]
y = concrete_new[TargetVar]

In [13]:
# Split the data into train and test (random sampling)

from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=143)

# Display the shape for train & test data

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((721, 8), (309, 8), (721,), (309,))

In [14]:
# Scaling the features by using MinMaxScaler

from sklearn.preprocessing import MinMaxScaler

mmscaler = MinMaxScaler(feature_range=(0, 1))

x_train = mmscaler.fit_transform(x_train)
x_train = pd.DataFrame(x_train)

x_test = mmscaler.fit_transform(x_test)
x_test = pd.DataFrame(x_test)

In [15]:
# Train the algorithm and build the model with train dataset

from sklearn.linear_model import LinearRegression

# Create an object for regression model

ModelRGR = LinearRegression()

# Train the model with training dataset

ModelRGR.fit(x_train, y_train)

# Predict the model with test dataset

y_pred = ModelRGR.predict(x_test)

# Evaluation metrics for Regression model

from sklearn import metrics

print('Mean Absolute Error (MAE):', round(metrics.mean_absolute_error(y_test, y_pred),3))    #here 3 is to end up with 3 decimal values

print('Mean Squared Error (MSE):', round(metrics.mean_squared_error(y_test, y_pred),3))  

print('Root Mean Squared Error (RMSE):', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)),3))

print('R2_score:', round(metrics.r2_score(y_test, y_pred),6)) 

print('Mean Absolute Percentage Error (MAPE):', round(metrics.mean_absolute_percentage_error(y_test, y_pred)*100,3), '%')

print('Root Mean Squared Log Error (RMSLE):', round(np.log(np.sqrt(metrics.mean_squared_error(y_test, y_pred))),3))


r_squared = round(metrics.r2_score(y_test, y_pred),6)
adjusted_r_squared = round(1 - (1-r_squared)*(len(y)-1)/(len(y)-x.shape[1]-1),6)

print('Adj R Square: ', adjusted_r_squared)

Mean Absolute Error (MAE): 8.386
Mean Squared Error (MSE): 115.568
Root Mean Squared Error (RMSE): 10.75
R2_score: 0.558885
Mean Absolute Percentage Error (MAPE): 31.327 %
Root Mean Squared Log Error (RMSLE): 2.375
Adj R Square:  0.555429


In [16]:
# Display the Final results 

Results = pd.DataFrame({'CSinMPa_A':y_test, 'CSinMPa_P':y_pred})

# Merge two Dataframes on index of both the dataframes

ResultsFinal = concrete_new.merge(Results, left_index=True, right_index=True)

# Display 10 records randomly

ResultsFinal.sample(10)

,Cement,Slag,Flyash,Water,SuperPlasticizer,CoarseAggregate,FineAggregate,Age,CSinMPa,CSinMPa_A,CSinMPa_P
616,277.0,0.0,0.0,191.0,0.0,968.0,856.0,360,33.70,33.70,55.875012
714,153.0,102.0,0.0,192.0,0.0,888.0,943.1,7,8.37,8.37,11.960261
533,289.0,0.0,0.0,192.0,0.0,913.2,895.3,3,11.65,11.65,14.641600
213,230.0,0.0,118.3,195.5,4.6,1029.4,758.6,100,35.34,35.34,29.769879
980,147.8,175.1,0.0,171.2,2.2,1000.0,828.5,28,26.92,26.92,27.230285
368,214.9,53.8,121.9,155.6,9.6,1014.3,780.6,100,56.63,56.63,43.740835
711,305.3,203.5,0.0,203.5,0.0,965.4,631.0,90,54.60,54.60,46.378928
105,401.8,94.7,0.0,147.4,11.4,946.8,852.1,7,54.10,54.10,52.092839
112,439.0,177.0,0.0,186.0,11.1,884.9,707.9,7,56.10,56.10,55.748697
464,167.0,75.4,167.0,164.0,7.9,1007.3,770.1,100,56.81,56.81,42.646857


In [17]:
ResultsFinal['%Error'] = round(((ResultsFinal['CSinMPa_A']-ResultsFinal['CSinMPa_P'])/ResultsFinal['CSinMPa_A'])*100,3)

In [18]:
del ResultsFinal['CSinMPa_A']

ResultsFinal.sample(10)

,Cement,Slag,Flyash,Water,SuperPlasticizer,CoarseAggregate,FineAggregate,Age,CSinMPa,CSinMPa_P,%Error
254,212.5,0.0,100.4,159.3,8.7,1007.8,903.6,3,13.54,24.814018,-83.265
1021,298.2,0.0,107.0,209.7,11.1,879.6,744.2,28,31.88,23.996338,24.729
356,277.2,97.8,24.5,160.7,11.2,1061.7,782.5,28,63.14,39.922409,36.772
832,147.0,115.0,89.0,202.0,9.0,860.0,829.0,28,19.99,20.830895,-4.207
570,295.8,0.0,0.0,185.7,0.0,1091.4,769.3,28,25.22,20.358924,19.275
238,213.8,98.1,24.5,181.7,6.7,1066.0,785.5,100,49.97,35.885134,28.187
894,143.0,169.0,143.0,191.0,8.0,967.0,643.0,28,29.72,32.172407,-8.252
788,349.0,0.0,0.0,192.0,0.0,1047.0,806.0,7,18.13,23.461758,-29.408
1007,155.6,243.5,0.0,180.3,10.7,1022.0,697.7,28,37.36,34.688478,7.151
166,375.0,93.8,0.0,126.6,23.4,852.1,992.6,91,62.50,66.322411,-6.116


# RGRResult

In [19]:
RGRResults = pd.read_csv(r"C:\Users\badda\Downloads\RGRResults (1).csv",header =0)
RGRResults.head()

,Model Name,Mean_Absolute_Error_MAE,Adj_R_Square,Root_Mean_Squared_Error_RMSE,Mean_Absolute_Percentage_Error_MAPE,Mean_Squared_Error_MSE,Root_Mean_Squared_Log_Error_RMSLE,R2_score


In [20]:
# Build the Regression / Regressor models

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

# Create objects of Regression / Regressor models with default hyper-parameters

ModelMLR = LinearRegression()
ModelDCR = DecisionTreeRegressor()
ModelRFR = RandomForestRegressor()
ModelETR = ExtraTreesRegressor()
ModelKNN = KNeighborsRegressor(n_neighbors=5)
ModelSVR = SVR()

# Evalution matrix for all the algorithms
 
MM = [ModelMLR, ModelDCR, ModelRFR, ModelETR, ModelKNN, ModelSVR]

for models in MM:
    
    # Fit the model with train data
    
    models.fit(x_train, y_train)
    
    # Predict the model with test data

    y_pred = models.predict(x_test)
    
    # Print the model name
    
    print('Model Name: ', models)
    
    # Evaluation metrics for Regression analysis

    from sklearn import metrics

    print('Mean Absolute Error (MAE):', round(metrics.mean_absolute_error(y_test, y_pred),3))  
    print('Mean Squared Error (MSE):', round(metrics.mean_squared_error(y_test, y_pred),3))  
    print('Root Mean Squared Error (RMSE):', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)),3))
    print('R2_score:', round(metrics.r2_score(y_test, y_pred),6))
    print('Root Mean Squared Log Error (RMSLE):', round(np.log(np.sqrt(metrics.mean_squared_error(y_test, y_pred))),3))
    
    # Define the function to calculate the MAPE - Mean Absolute Percentage Error

    def MAPE (y_test, y_pred):
        y_test, y_pred = np.array(y_test), np.array(y_pred)
        return np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    
    # Evaluation of MAPE 

    result = MAPE(y_test, y_pred)
    print('Mean Absolute Percentage Error (MAPE):', round(result, 2), '%')
    
    # Calculate Adjusted R squared values 

    r_squared = round(metrics.r2_score(y_test, y_pred),6)
    adjusted_r_squared = round(1 - (1-r_squared)*(len(y)-1)/(len(y)-x.shape[1]-1),6)
    print('Adj R Square: ', adjusted_r_squared)
    print('------------------------------------------------------------------------------------------------------------')
    #-------------------------------------------------------------------------------------------
    new_row = {'Model Name' : models,
               'Mean_Absolute_Error_MAE' : metrics.mean_absolute_error(y_test, y_pred),
               'Adj_R_Square' : adjusted_r_squared,
               'Root_Mean_Squared_Error_RMSE' : np.sqrt(metrics.mean_squared_error(y_test, y_pred)),
               'Mean_Absolute_Percentage_Error_MAPE' : result,
               'Mean_Squared_Error_MSE' : metrics.mean_squared_error(y_test, y_pred),
               'Root_Mean_Squared_Log_Error_RMSLE': np.log(np.sqrt(metrics.mean_squared_error(y_test, y_pred))),
               'R2_score' : metrics.r2_score(y_test, y_pred)}
    RGRResults = RGRResults._append(new_row, ignore_index=True)

Model Name:  LinearRegression()
Mean Absolute Error (MAE): 8.386
Mean Squared Error (MSE): 115.568
Root Mean Squared Error (RMSE): 10.75
R2_score: 0.558885
Root Mean Squared Log Error (RMSLE): 2.375
Mean Absolute Percentage Error (MAPE): 31.33 %
Adj R Square:  0.555429
------------------------------------------------------------------------------------------------------------
Model Name:  DecisionTreeRegressor()
Mean Absolute Error (MAE): 6.117
Mean Squared Error (MSE): 77.917
Root Mean Squared Error (RMSE): 8.827
R2_score: 0.702596
Root Mean Squared Log Error (RMSLE): 2.178
Mean Absolute Percentage Error (MAPE): 19.48 %
Adj R Square:  0.700266
------------------------------------------------------------------------------------------------------------
Model Name:  RandomForestRegressor()
Mean Absolute Error (MAE): 4.769
Mean Squared Error (MSE): 41.792
Root Mean Squared Error (RMSE): 6.465
R2_score: 0.840483
Root Mean Squared Log Error (RMSLE): 1.866
Mean Absolute Percentage Error (MAP

In [21]:
RGRResults.head(10)

,Model Name,Mean_Absolute_Error_MAE,Adj_R_Square,Root_Mean_Squared_Error_RMSE,Mean_Absolute_Percentage_Error_MAPE,Mean_Squared_Error_MSE,Root_Mean_Squared_Log_Error_RMSLE,R2_score
0,LinearRegression(),8.385699,0.555429,10.750258,31.326908,115.568039,2.374930,0.558885
1,DecisionTreeRegressor(),6.117201,0.700266,8.827059,19.478517,77.916964,2.177822,0.702596
2,"(DecisionTreeRegressor(max_features=1.0, rando...",4.768831,0.839233,6.464675,14.785957,41.792026,1.866353,0.840483
3,"(ExtraTreeRegressor(random_state=1753271828), ...",4.388441,0.853961,6.161461,13.517889,37.963601,1.818314,0.855096
4,KNeighborsRegressor(),7.276524,0.650777,9.527948,26.921693,90.781790,2.254229,0.653492
5,SVR(),8.445729,0.537147,10.969064,32.725504,120.320371,2.395079,0.540745


In [23]:
y_predf = ModelSVR.predict(x_test)

In [25]:
# Display the Final results 

Results = pd.DataFrame({'CSinMPa_A':y_test, 'CSinMPa_P':y_predf})

# Merge two Dataframes on index of both the dataframes

ResultsFinal = concrete_new.merge(Results, left_index=True, right_index=True)

# Display 10 records randomly

ResultsFinal.sample(10)

,Cement,Slag,Flyash,Water,SuperPlasticizer,CoarseAggregate,FineAggregate,Age,CSinMPa,CSinMPa_A,CSinMPa_P
304,277.1,0.0,97.4,160.6,11.8,973.9,875.6,3,23.14,23.14,31.925326
577,135.7,203.5,0.0,185.7,0.0,1076.2,759.3,28,18.20,18.20,25.564455
982,312.7,0.0,0.0,178.1,8.0,999.7,822.2,28,25.10,25.10,30.664625
495,387.0,20.0,94.0,157.0,13.9,938.0,845.0,56,47.97,47.97,49.195484
886,148.0,0.0,137.0,158.0,16.0,1002.0,830.0,28,17.95,17.95,27.974939
438,167.0,75.4,167.0,164.0,7.9,1007.3,770.1,28,41.41,41.41,28.875272
811,310.0,0.0,0.0,192.0,0.0,970.0,850.0,28,27.92,27.92,22.730982
641,250.0,0.0,0.0,182.0,0.0,1100.0,820.0,7,12.73,12.73,16.630328
151,401.8,94.7,0.0,147.4,11.4,946.8,852.1,56,73.70,73.70,56.053456
743,397.0,0.0,0.0,186.0,0.0,1040.0,734.0,28,36.94,36.94,32.337332
